In [1]:
import os

import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

# Set global variables

In [2]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
DATA_PATH =  "../input/data-jigsaw1"
SEQUENCE_LENGTH = 256
MODEL = 'jplu/tf-xlm-roberta-large'
OUTPUT_PATH = "/kaggle/working"

In [3]:
def get_tokenizer():
    """Obtenez le tokenizer pour XLM-Roberta"""
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
  
    return tokenizer

tokenizer = get_tokenizer()

In [4]:
train = pd.read_csv(DATA_PATH + "/train.csv")
valid = pd.read_csv(DATA_PATH + "/validation.csv")
test = pd.read_csv(DATA_PATH + "/test.csv")

# Preprocessing

In [5]:
def encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_masks=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'])

In [6]:
%%time 

x_train = encode(train.comment_text.values, tokenizer, maxlen=SEQUENCE_LENGTH)
x_valid = encode(valid.comment_text.values, tokenizer, maxlen=SEQUENCE_LENGTH)
x_test = encode(test.comment_text.values, tokenizer, maxlen=SEQUENCE_LENGTH)

CPU times: user 4min 31s, sys: 3.6 s, total: 4min 35s
Wall time: 4min 35s


In [7]:
np.save('x_train', x_train)
np.save('x_valid', x_valid)
np.save('x_test', x_test)

np.save('y_train', train.toxic.values)
np.save('y_valid', valid.toxic.values)